 # Task 2: Predictive Analysis Using Machine Learning

## we will follow an optimized approach using Dask for preprocessing and Pandas for ML modeling.

In [1]:
!pip show dask

Name: dask
Version: 2024.5.0
Summary: Parallel PyData with Task Scheduling
Home-page: https://github.com/dask/dask/
Author: 
Author-email: 
License: BSD-3-Clause
Location: C:\Users\Deepti\AppData\Roaming\Python\Python312\site-packages
Requires: click, cloudpickle, fsspec, packaging, partd, pyyaml, toolz
Required-by: dask-expr, dask-glm, dask-ml, datashader, distributed, intake


In [2]:
try:
    import os
    import json
    import math
    import dask
    from dask.distributed import Client
    import dask.dataframe as dd
    import numpy as np
    import dask.multiprocessing
except Exception as e:
    print("Some Modules are Missing : {} ".format(e))

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'cleaned_sales_data.csv',
 'sales_5000000.csv',
 'Task 2.ipynb',
 'Task1.ipynb']

In [4]:
import dask.dataframe as dd

In [5]:
sales= dd.read_csv(r'C:\Users\Deepti\Desktop\study folder\internship\Task\sales_5000000.csv')

In [6]:
print(sales.head())

                         Region  Country        Item Type Sales Channel  \
0         Australia and Oceania    Palau  Office Supplies        Online   
1                        Europe   Poland        Beverages        Online   
2                 North America   Canada           Cereal        Online   
3                        Europe  Belarus           Snacks        Online   
4  Middle East and North Africa     Oman           Cereal       Offline   

  Order Priority  Order Date   Order ID   Ship Date  Units Sold  Unit Price  \
0              H  2020-03-06  517073523  2020-03-26        2401      651.21   
1              L  2014-04-18  380507028  2014-05-26        9340       47.45   
2              M  2019-01-08  504055583  2019-01-31         103      205.70   
3              C  2018-01-19  954955518  2018-02-27        1414      152.58   
4              H  2023-04-26  970755660  2023-06-02        7027      205.70   

   Unit Cost  Total Revenue  Total Cost  Total Profit  
0     524.96     1

In [7]:
print(sales.dtypes)

Region            string[pyarrow]
Country           string[pyarrow]
Item Type         string[pyarrow]
Sales Channel     string[pyarrow]
Order Priority    string[pyarrow]
Order Date        string[pyarrow]
Order ID                    int64
Ship Date         string[pyarrow]
Units Sold                  int64
Unit Price                float64
Unit Cost                 float64
Total Revenue             float64
Total Cost                float64
Total Profit              float64
dtype: object


In [8]:
print("Rows:", sales.shape[0].compute())

Rows: 5000000


In [9]:
sales.isnull().sum().compute()

Region            0
Country           0
Item Type         0
Sales Channel     0
Order Priority    0
Order Date        0
Order ID          0
Ship Date         0
Units Sold        0
Unit Price        0
Unit Cost         0
Total Revenue     0
Total Cost        0
Total Profit      0
dtype: int64

In [10]:
sales.nunique().compute()

Region                  7
Country               185
Item Type              12
Sales Channel           2
Order Priority          4
Order Date           3906
Order ID          2097152
Ship Date            3956
Units Sold          10000
Unit Price             12
Unit Cost              12
Total Revenue      119953
Total Cost         119786
Total Profit       119831
dtype: int64

In [11]:
sales['Order ID'].value_counts().compute()[sales['Order ID'].value_counts().compute() > 1].count()


np.int64(2043424)

In [12]:
sales['Order ID'].nunique().compute() == len(sales)

np.False_

In [13]:
filtered_rows = sales[sales["Order ID"] == 100005042].compute()
print(filtered_rows)

                    Region Country        Item Type Sales Channel  \
358743  Sub-Saharan Africa    Togo  Office Supplies       Offline   
301955  Sub-Saharan Africa    Togo  Office Supplies       Offline   
282551  Sub-Saharan Africa    Togo  Office Supplies       Offline   

       Order Priority  Order Date   Order ID   Ship Date  Units Sold  \
358743              C  2015-12-05  100005042  2016-01-12        2569   
301955              C  2015-12-05  100005042  2016-01-12        2569   
282551              C  2015-12-05  100005042  2016-01-12        2569   

        Unit Price  Unit Cost  Total Revenue  Total Cost  Total Profit  
358743      651.21     524.96     1672958.49  1348622.24     324336.25  
301955      651.21     524.96     1672958.49  1348622.24     324336.25  
282551      651.21     524.96     1672958.49  1348622.24     324336.25  


In [14]:
sales = sales.drop_duplicates(subset=["Order ID"], keep="first")


In [15]:
duplicate_count = sales.groupby("Order ID").size().compute()
print((duplicate_count > 1).sum())  # Should print 0 if no duplicates exist


0


In [16]:
filtered_rows = sales[sales["Order ID"] == 100005042]  # No .compute() yet
print(filtered_rows.compute())  # Compute only when needed


                    Region Country        Item Type Sales Channel  \
358743  Sub-Saharan Africa    Togo  Office Supplies       Offline   

       Order Priority  Order Date   Order ID   Ship Date  Units Sold  \
358743              C  2015-12-05  100005042  2016-01-12        2569   

        Unit Price  Unit Cost  Total Revenue  Total Cost  Total Profit  
358743      651.21     524.96     1672958.49  1348622.24     324336.25  


In [18]:
print(len(sales))  # Displays the count of remaining rows

2097152


In [19]:
sales = sales.reset_index(drop=True)
print(sales.head())

                         Region     Country        Item Type Sales Channel  \
0  Middle East and North Africa       Yemen        Household        Online   
1                          Asia     Myanmar       Vegetables       Offline   
2            Sub-Saharan Africa  The Gambia  Office Supplies        Online   
3            Sub-Saharan Africa      Malawi           Snacks        Online   
4  Middle East and North Africa        Oman       Vegetables        Online   

  Order Priority  Order Date   Order ID   Ship Date  Units Sold  Unit Price  \
0              C  2019-04-18  479070270  2019-05-08        6394      668.27   
1              H  2020-03-28  727876698  2020-04-19        3745      154.06   
2              L  2015-01-28  958184897  2015-03-07        7059      651.21   
3              M  2017-08-08  690064418  2017-08-29        4251      152.58   
4              L  2016-06-02  697118413  2016-07-16        1685      154.06   

   Unit Cost  Total Revenue  Total Cost  Total Profit  


In [20]:
print(sales.dtypes)

Region            string[pyarrow]
Country           string[pyarrow]
Item Type         string[pyarrow]
Sales Channel     string[pyarrow]
Order Priority    string[pyarrow]
Order Date        string[pyarrow]
Order ID                    int64
Ship Date         string[pyarrow]
Units Sold                  int64
Unit Price                float64
Unit Cost                 float64
Total Revenue             float64
Total Cost                float64
Total Profit              float64
dtype: object


In [21]:
sales["Order Date"] = sales["Order Date"].astype("datetime64[ns]")
sales["Ship Date"] = sales["Ship Date"].astype("datetime64[ns]")

In [22]:
print(sales.dtypes)

Region            string[pyarrow]
Country           string[pyarrow]
Item Type         string[pyarrow]
Sales Channel     string[pyarrow]
Order Priority    string[pyarrow]
Order Date         datetime64[ns]
Order ID                    int64
Ship Date          datetime64[ns]
Units Sold                  int64
Unit Price                float64
Unit Cost                 float64
Total Revenue             float64
Total Cost                float64
Total Profit              float64
dtype: object


In [23]:
sales.describe().compute()

,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
count,2097152,2.097152e+06,2097152,2.097152e+06,2.097152e+06,2.097152e+06,2.097152e+06,2.097152e+06,2.097152e+06
min,2014-01-01 00:00:00,1.000003e+08,2014-01-01 00:00:00,1.000000e+00,9.330000e+00,6.920000e+00,9.330000e+00,6.920000e+00,2.410000e+00
25%,2016-09-08 00:00:00,3.265730e+08,2016-10-03 00:00:00,2.519000e+03,1.092800e+02,5.667000e+01,2.803346e+05,1.634930e+05,9.572076e+04
50%,2019-05-15 00:00:00,5.514017e+08,2019-06-08 00:00:00,5.016000e+03,2.057000e+02,1.171100e+02,7.891109e+05,4.701480e+05,2.844007e+05
75%,2022-01-15 00:00:00,7.757283e+08,2022-02-08 00:00:00,7.519000e+03,4.372000e+02,3.646900e+02,1.829592e+06,1.204936e+06,5.671582e+05
max,2024-09-10 00:00:00,9.999999e+08,2024-10-30 00:00:00,1.000000e+04,6.682700e+02,5.249600e+02,6.682700e+06,5.249600e+06,1.738700e+06
mean,NaN,5.500001e+08,NaN,5.000498e+03,2.661651e+02,1.876368e+02,1.331001e+06,9.383151e+05,3.926858e+05
std,NaN,2.598077e+08,NaN,2.886752e+03,2.170110e+02,1.756957e+02,1.469895e+06,1.150072e+06,3.791404e+05


In [24]:
# Check for missing values in each column
missing_values = sales.isnull().sum().compute()

# Display columns with missing values
print("Missing Values in Each Column:\n", missing_values)


Missing Values in Each Column:
 Region            0
Country           0
Item Type         0
Sales Channel     0
Order Priority    0
Order Date        0
Order ID          0
Ship Date         0
Units Sold        0
Unit Price        0
Unit Cost         0
Total Revenue     0
Total Cost        0
Total Profit      0
dtype: int64


In [25]:
# Compute the final cleaned dataset before saving
sales = sales.compute()

# Save preprocessed data for ML modeling
sales.to_csv("cleaned_sales_data.csv", index=False)

print("✅ Preprocessing Completed! Data is ready for ML modeling.")

✅ Preprocessing Completed! Data is ready for ML modeling.


In [26]:
sales.shape

(2097152, 14)

In [27]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2097152 entries, 0 to 233250
Data columns (total 14 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Region          object        
 1   Country         object        
 2   Item Type       object        
 3   Sales Channel   object        
 4   Order Priority  object        
 5   Order Date      datetime64[ns]
 6   Order ID        int64         
 7   Ship Date       datetime64[ns]
 8   Units Sold      int64         
 9   Unit Price      float64       
 10  Unit Cost       float64       
 11  Total Revenue   float64       
 12  Total Cost      float64       
 13  Total Profit    float64       
dtypes: datetime64[ns](2), float64(5), int64(2), object(5)
memory usage: 240.0+ MB


#### import libraries

In [28]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb

#### Feature engineering

In [29]:
sales['Delivery Days'] = (sales['Ship Date'] - sales['Order Date']).dt.days  # Delivery Time Calculation


#### Encoding categorical variable

In [30]:
categorical_cols = ['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    sales[col] = sales[col].astype(str)  # Convert to string
    sales[col] = le.fit_transform(sales[col])
    label_encoders[col] = le  # Store encoders for later use

#### features and target variable

In [31]:
features = ['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority', 'Units Sold', 'Unit Price', 'Unit Cost']
X_revenue = sales[features]
y_revenue = sales['Total Revenue']

X_delivery = sales[features]
y_delivery = sales['Delivery Days']

#### train and test split

In [32]:
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_revenue, y_revenue, test_size=0.2, random_state=42)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_delivery, y_delivery, test_size=0.2, random_state=42)

#### model training

In [33]:
model_revenue = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model_revenue.fit(X_train_r, y_train_r)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [35]:
model_delivery = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, random_state=42)
model_delivery.fit(X_train_d, y_train_d)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

#### model predction

In [39]:
sales['Order Month'] = sales['Order Date'].dt.month
sales['Order Weekday'] = sales['Order Date'].dt.weekday
sales['Processing Time'] = (sales['Ship Date'] - sales['Order Date']).dt.days

In [36]:
y_pred_r = model_revenue.predict(X_test_r)  # Predicted Revenue
y_pred_d = model_delivery.predict(X_test_d)  # Predicted Delivery Days

#### model eveluation

In [37]:
print("Revenue Prediction Model:")
print("MAE:", mean_absolute_error(y_test_r, y_pred_r))
print("R² Score:", r2_score(y_test_r, y_pred_r))

print("\nDelivery Time Prediction Model:")
print("MAE:", mean_absolute_error(y_test_d, y_pred_d))
print("R² Score:", r2_score(y_test_d, y_pred_d))

Revenue Prediction Model:
MAE: 4068.9723920519873
R² Score: 0.9999860059715912

Delivery Time Prediction Model:
MAE: 12.692132949829102
R² Score: 0.0069141387939453125


#### save preduction file

In [38]:
df_predicted = pd.DataFrame({
    'Actual Revenue': y_test_r,
    'Predicted Revenue': y_pred_r,
    'Actual Delivery Days': y_test_d,
    'Predicted Delivery Days': y_pred_d
})

df_predicted.to_csv("predicted_results.csv", index=False)

In [43]:
sales['Order Month'] = sales['Order Date'].dt.month
sales['Order Weekday'] = sales['Order Date'].dt.weekday
sales['Order Year'] = sales['Order Date'].dt.year
sales['Processing Time'] = (sales['Ship Date'] - sales['Order Date']).dt.days  # Already calculated

In [44]:
features = ['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority', 
            'Units Sold', 'Unit Price', 'Unit Cost', 'Order Month', 'Order Weekday', 'Processing Time']
X_delivery = sales[features]  
y_delivery = sales['Delivery Days']

In [46]:
X_delivery.loc[:, ['Units Sold', 'Unit Price', 'Unit Cost', 'Processing Time']] = scaler.fit_transform(
    X_delivery[['Units Sold', 'Unit Price', 'Unit Cost', 'Processing Time']]
)

In [47]:
from sklearn.model_selection import train_test_split

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_delivery, y_delivery, test_size=0.2, random_state=42)

In [48]:
from xgboost import XGBRegressor

model_delivery = XGBRegressor(n_estimators=500,  
                              learning_rate=0.05,  
                              max_depth=8,  
                              min_child_weight=5,  
                              subsample=0.8,  
                              colsample_bytree=0.8,  
                              random_state=42)

model_delivery.fit(X_train_d, y_train_d)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=5, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, ...)

In [49]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred_d = model_delivery.predict(X_test_d)

print("Delivery Time Prediction Model:")
print("MAE:", mean_absolute_error(y_test_d, y_pred_d))
print("R² Score:", r2_score(y_test_d, y_pred_d))

Delivery Time Prediction Model:
MAE: 0.030102286487817764
R² Score: 0.9999905824661255


In [50]:
print("Revenue Prediction Model:")
print("MAE:", mean_absolute_error(y_test_r, y_pred_r))
print("R² Score:", r2_score(y_test_r, y_pred_r))

Revenue Prediction Model:
MAE: 4068.9723920519873
R² Score: 0.9999860059715912


### Insights 

#### 1. Revenue Prediction Insight

#### 2.Delivery time Prediction Insight

## Overall